In [35]:
import pandas as pd
import numpy as np

In [36]:
file_path = './data/'

# read_data
user = pd.read_csv(file_path + 'users.csv')
user_depoist = pd.read_csv(file_path + 'user_deposit.csv')
user_savings = pd.read_csv(file_path + 'user_savings.csv')
depositproducts = pd.read_csv(file_path + 'depositproducts.csv')
savingsproducts = pd.read_csv(file_path + 'installmentproducts.csv')


In [5]:
features = pd.DataFrame({'user_id':user_depoist.user_id.unique()})
features

,user_id
0,1
1,2
2,3
3,4
4,5
...,...
9995,9996
9996,9997
9997,9998
9998,9999


In [6]:
# 가입한 예금 상품 수
f = user_depoist.groupby('user_id')['deposit_id'].agg([('예금가입상품수',np.size)]).reset_index()
features = features.merge(f, how='left')
features

,user_id,예금가입상품수
0,1,2
1,2,2
2,3,5
3,4,5
4,5,5
...,...,...
9995,9996,4
9996,9997,5
9997,9998,2
9998,9999,4


In [7]:
# 가입한 적금 상품수
f = user_savings.groupby('user_id')['savings_id'].agg([('적금가입상품수',np.size)]).reset_index()
features = features.merge(f, how='left')
features

,user_id,예금가입상품수,적금가입상품수
0,1,2,4
1,2,2,2
2,3,5,6
3,4,5,3
4,5,5,4
...,...,...,...
9995,9996,4,6
9996,9997,5,5
9997,9998,2,2
9998,9999,4,6


In [8]:
# 주거래은행
df_1 = pd.merge(user_depoist, depositproducts, left_on='deposit_id', right_on='id', how='left')[['user_id','kor_co_nm']]
df_2 = pd.merge(user_savings, savingsproducts, left_on='savings_id', right_on='id', how='left')[['user_id','kor_co_nm']]
df = pd.concat([df_1,df_2])
f = df.groupby('user_id')['kor_co_nm'].agg([('주거래은행', lambda x:x.value_counts().index[0])]).reset_index()

features = features.merge(f, how='left')
features

,user_id,예금가입상품수,적금가입상품수,주거래은행
0,1,2,4,중소기업은행
1,2,2,2,농협은행주식회사
2,3,5,6,농협은행주식회사
3,4,5,3,농협은행주식회사
4,5,5,4,부산은행
...,...,...,...,...
9995,9996,4,6,전북은행
9996,9997,5,5,아이엠뱅크
9997,9998,2,2,토스뱅크 주식회사
9998,9999,4,6,농협은행주식회사


In [9]:
user = user[['id', 'investment_style', 'annual_income', 'savings_goal','age','gender']]


In [10]:
features = pd.merge(features, user, left_on='user_id', right_on='id', how='left')
features

,user_id,예금가입상품수,적금가입상품수,주거래은행,id,investment_style,annual_income,savings_goal,age,gender
0,1,2,4,중소기업은행,1,moderate,2,1,42,M
1,2,2,2,농협은행주식회사,2,aggressive,0,0,25,M
2,3,5,6,농협은행주식회사,3,moderate,5,2,49,F
3,4,5,3,농협은행주식회사,4,aggressive,4,2,23,F
4,5,5,4,부산은행,5,conservative,4,3,20,M
...,...,...,...,...,...,...,...,...,...,...
9995,9996,4,6,전북은행,9996,aggressive,0,3,28,M
9996,9997,5,5,아이엠뱅크,9997,aggressive,1,4,20,F
9997,9998,2,2,토스뱅크 주식회사,9998,conservative,3,2,44,M
9998,9999,4,6,농협은행주식회사,9999,conservative,3,1,48,F


In [11]:
def change_investment_style(x):
    if x == 'moderate':
        return 0
    elif x == 'aggressive':
        return 1
    else:
        return 2

In [12]:
features['investment_style'] = features['investment_style'].apply(change_investment_style)
features

,user_id,예금가입상품수,적금가입상품수,주거래은행,id,investment_style,annual_income,savings_goal,age,gender
0,1,2,4,중소기업은행,1,0,2,1,42,M
1,2,2,2,농협은행주식회사,2,1,0,0,25,M
2,3,5,6,농협은행주식회사,3,0,5,2,49,F
3,4,5,3,농협은행주식회사,4,1,4,2,23,F
4,5,5,4,부산은행,5,2,4,3,20,M
...,...,...,...,...,...,...,...,...,...,...
9995,9996,4,6,전북은행,9996,1,0,3,28,M
9996,9997,5,5,아이엠뱅크,9997,1,1,4,20,F
9997,9998,2,2,토스뱅크 주식회사,9998,2,3,2,44,M
9998,9999,4,6,농협은행주식회사,9999,2,3,1,48,F


In [13]:
def change_gender(x):
    if x == 'M':
        return 0
    else:
        return 1

In [14]:
features['gender'] = features['gender'].apply(change_gender)
features

,user_id,예금가입상품수,적금가입상품수,주거래은행,id,investment_style,annual_income,savings_goal,age,gender
0,1,2,4,중소기업은행,1,0,2,1,42,0
1,2,2,2,농협은행주식회사,2,1,0,0,25,0
2,3,5,6,농협은행주식회사,3,0,5,2,49,1
3,4,5,3,농협은행주식회사,4,1,4,2,23,1
4,5,5,4,부산은행,5,2,4,3,20,0
...,...,...,...,...,...,...,...,...,...,...
9995,9996,4,6,전북은행,9996,1,0,3,28,0
9996,9997,5,5,아이엠뱅크,9997,1,1,4,20,1
9997,9998,2,2,토스뱅크 주식회사,9998,2,3,2,44,0
9998,9999,4,6,농협은행주식회사,9999,2,3,1,48,1


In [15]:
def change_mainbank(x):
    if x == '중소기업은행':
        return 0
    elif x == '농협은행주식회사':
        return 1
    elif x == '부산은행':
        return 2
    elif x == '경남은행':
        return 3
    elif x == '수협은행':
        return 4
    elif x == '한국스탠다드차타드은행':
        return 5
    elif x == '광주은행':
        return 6
    elif x == '국민은행':
        return 7
    elif x == '제주은행':
        return 8
    elif x == '아이엠뱅크':
        return 9
    elif x == '토스뱅크 주식회사':
        return 10
    elif x == '한국산업은행':
        return 11
    elif x == '주식회사 케이뱅크':
        return 12
    elif x == '전북은행':
        return 13
    elif x == '신한은행':
        return 14
    elif x == '우리은행':
        return 15
    elif x == '하나은행':
        return 16
    else:
        return 17

In [16]:
features['주거래은행'] = features['주거래은행'].apply(change_mainbank)
features = features[['user_id','예금가입상품수','적금가입상품수','주거래은행','investment_style','annual_income','savings_goal','age','gender']]
features


,user_id,예금가입상품수,적금가입상품수,주거래은행,investment_style,annual_income,savings_goal,age,gender
0,1,2,4,0,0,2,1,42,0
1,2,2,2,1,1,0,0,25,0
2,3,5,6,1,0,5,2,49,1
3,4,5,3,1,1,4,2,23,1
4,5,5,4,2,2,4,3,20,0
...,...,...,...,...,...,...,...,...,...
9995,9996,4,6,13,1,0,3,28,0
9996,9997,5,5,9,1,1,4,20,1
9997,9998,2,2,10,2,3,2,44,0
9998,9999,4,6,1,2,3,1,48,1


In [17]:
features

,user_id,예금가입상품수,적금가입상품수,주거래은행,investment_style,annual_income,savings_goal,age,gender
0,1,2,4,0,0,2,1,42,0
1,2,2,2,1,1,0,0,25,0
2,3,5,6,1,0,5,2,49,1
3,4,5,3,1,1,4,2,23,1
4,5,5,4,2,2,4,3,20,0
...,...,...,...,...,...,...,...,...,...
9995,9996,4,6,13,1,0,3,28,0
9996,9997,5,5,9,1,1,4,20,1
9997,9998,2,2,10,2,3,2,44,0
9998,9999,4,6,1,2,3,1,48,1


In [18]:
features.to_csv('./data/features.csv', index=False)

In [34]:
features = pd.read_csv(file_path + 'features.csv')
features


,user_id,예금가입상품수,적금가입상품수,주거래은행,investment_style,annual_income,savings_goal,age,gender
0,1,2,4,0,0,2,1,42,0
1,2,2,2,1,1,0,0,25,0
2,3,5,6,1,0,5,2,49,1
3,4,5,3,1,1,4,2,23,1
4,5,5,4,2,2,4,3,20,0
...,...,...,...,...,...,...,...,...,...
9995,9996,4,6,13,1,0,3,28,0
9996,9997,5,5,9,1,1,4,20,1
9997,9998,2,2,10,2,3,2,44,0
9998,9999,4,6,1,2,3,1,48,1


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [25]:
data = {
    'user_id':1000000,
    '예금가입상품수':2,
    '적금가입상품수':4,
    '주거래은행':'국민은행',
    'investment_style':0,
    'annual_income':1,
    'savings_goal':1,
    'age':20,
    'gender':'M'
}


In [26]:
def recommend_system(data):
    print(data)

In [27]:
recommend_system(data)

{'user_id': 1000000, '예금가입상품수': 2, '적금가입상품수': 4, '주거래은행': '국민은행', 'investment_style': 0, 'annual_income': 1, 'savings_goal': 1, 'age': 20, 'gender': 'M'}


In [42]:
features = pd.read_csv(file_path+'features.csv')

In [43]:
features

,user_id,예금가입상품수,적금가입상품수,주거래은행,investment_style,annual_income,savings_goal,age,gender
0,1,2,4,0,0,2,1,42,0
1,2,2,2,1,1,0,0,25,0
2,3,5,6,1,0,5,2,49,1
3,4,5,3,1,1,4,2,23,1
4,5,5,4,2,2,4,3,20,0
...,...,...,...,...,...,...,...,...,...
9995,9996,4,6,13,1,0,3,28,0
9996,9997,5,5,9,1,1,4,20,1
9997,9998,2,2,10,2,3,2,44,0
9998,9999,4,6,1,2,3,1,48,1


In [44]:
new_data = pd.DataFrame([data])
# new_data.set_index('user_id', inplace=True)

In [45]:
new_data

,user_id,예금가입상품수,적금가입상품수,주거래은행,investment_style,annual_income,savings_goal,age,gender
0,1000000,2,4,국민은행,0,1,1,20,M


In [46]:
df = pd.concat([features, new_data])

In [47]:
df

,user_id,예금가입상품수,적금가입상품수,주거래은행,investment_style,annual_income,savings_goal,age,gender
0,1,2,4,0,0,2,1,42,0
1,2,2,2,1,1,0,0,25,0
2,3,5,6,1,0,5,2,49,1
3,4,5,3,1,1,4,2,23,1
4,5,5,4,2,2,4,3,20,0
...,...,...,...,...,...,...,...,...,...
9996,9997,5,5,9,1,1,4,20,1
9997,9998,2,2,10,2,3,2,44,0
9998,9999,4,6,1,2,3,1,48,1
9999,10000,1,5,3,0,5,3,26,0


In [48]:
df.set_index('user_id',inplace=True)

In [49]:
df

,예금가입상품수,적금가입상품수,주거래은행,investment_style,annual_income,savings_goal,age,gender
user_id,,,,,,,,
1,2,4,0,0,2,1,42,0
2,2,2,1,1,0,0,25,0
3,5,6,1,0,5,2,49,1
4,5,3,1,1,4,2,23,1
5,5,4,2,2,4,3,20,0
...,...,...,...,...,...,...,...,...
9997,5,5,9,1,1,4,20,1
9998,2,2,10,2,3,2,44,0
9999,4,6,1,2,3,1,48,1


In [53]:
new_data['investment_style'] = new_data['investment_style'].apply(change_investment_style)
new_data['gender'] = new_data['gender'].apply(change_gender)
new_data['주거래은행'] = new_data['주거래은행'].apply(change_mainbank)

In [54]:
new_data

,user_id,예금가입상품수,적금가입상품수,주거래은행,investment_style,annual_income,savings_goal,age,gender
0,1000000,2,4,7,2,1,1,20,0


In [56]:
df = pd.concat([features,new_data])
df.set_index('user_id',inplace=True)
df

,예금가입상품수,적금가입상품수,주거래은행,investment_style,annual_income,savings_goal,age,gender
user_id,,,,,,,,
1,2,4,0,0,2,1,42,0
2,2,2,1,1,0,0,25,0
3,5,6,1,0,5,2,49,1
4,5,3,1,1,4,2,23,1
5,5,4,2,2,4,3,20,0
...,...,...,...,...,...,...,...,...
9997,5,5,9,1,1,4,20,1
9998,2,2,10,2,3,2,44,0
9999,4,6,1,2,3,1,48,1


In [79]:
import time
start = time.time()
from sklearn.metrics.pairwise import cosine_similarity
user_sim = cosine_similarity(df, df)
user_sim = pd.DataFrame(user_sim, df.index, df.index)
user_sim

user_id,1,2,3,4,5,6,7,8,9,10,...,9992,9993,9994,9995,9996,9997,9998,9999,10000,1000000
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.996397,0.996275,0.974713,0.952369,0.980212,0.984019,0.996777,0.996491,0.971937,...,0.993559,0.995482,0.858120,0.894532,0.898682,0.883116,0.972820,0.997584,0.976204,0.936179
2,0.996397,1.000000,0.991897,0.972118,0.951469,0.979166,0.986921,0.997148,0.997314,0.976487,...,0.994552,0.993534,0.875055,0.911471,0.914274,0.896825,0.979021,0.996511,0.966845,0.948646
3,0.996275,0.991897,1.000000,0.989512,0.972355,0.982089,0.987947,0.988448,0.996317,0.983021,...,0.997336,0.994435,0.869882,0.906194,0.909650,0.903531,0.973648,0.998048,0.985455,0.944681
4,0.974713,0.972118,0.989512,1.000000,0.992347,0.974778,0.984902,0.960349,0.982458,0.988032,...,0.988785,0.976925,0.875774,0.908883,0.910239,0.921237,0.960895,0.983902,0.979213,0.942021
5,0.952369,0.951469,0.972355,0.992347,1.000000,0.965720,0.973550,0.934763,0.962285,0.988569,...,0.974292,0.960708,0.885870,0.923892,0.924136,0.942051,0.951042,0.966668,0.976024,0.950475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9997,0.883116,0.896825,0.903531,0.921237,0.942051,0.926155,0.927888,0.881683,0.904832,0.961098,...,0.923061,0.915171,0.972622,0.991149,0.990415,1.000000,0.942064,0.900900,0.927955,0.978232
9998,0.972820,0.979021,0.973648,0.960895,0.951042,0.984434,0.987530,0.978532,0.980400,0.985029,...,0.984083,0.988591,0.946679,0.959138,0.961043,0.942064,1.000000,0.975560,0.974070,0.981371
9999,0.997584,0.996511,0.998048,0.983902,0.966668,0.980425,0.986984,0.992472,0.996489,0.981509,...,0.997847,0.994450,0.867471,0.908228,0.911295,0.900900,0.975560,1.000000,0.980187,0.948692


In [80]:
user_sim[1].sort_values(ascending=False)[1:20].reset_index()

,user_id,1
0,2889,0.999601
1,2607,0.999539
2,5526,0.999515
3,2460,0.999441
4,3432,0.999313
5,6644,0.999311
6,1263,0.999272
7,1395,0.999241
8,9578,0.999218
9,1502,0.999175


In [94]:
list(user_sim[1000000].sort_values(ascending=False)[1:20].reset_index().user_id)

[9592,
 2144,
 234,
 6475,
 6643,
 5107,
 1377,
 6688,
 6868,
 8189,
 3536,
 4846,
 7675,
 1815,
 1595,
 937,
 6328,
 1060,
 9845]